<a href="https://colab.research.google.com/github/vanryuji/sqlalchemy/blob/master/tutorial/sqlalchemy_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install sqlalchemy

In [0]:
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String
from sqlalchemy.orm import sessionmaker, scoped_session
from sqlalchemy import ForeignKey
from sqlalchemy.orm import relationship

In [0]:
### Connecting ###

engine = create_engine('sqlite:///:memory:', echo=True)

In [5]:
### Creating session ###

session = scoped_session(sessionmaker(bind=engine))
print('autocommit:', session.autocommit)

autocommit: False


In [0]:
### Declare mapping ###

Base = declarative_base()

class User(Base):
    __tablename__ = 'users'
    
    id = Column(Integer, primary_key=True)
    name = Column(String)
    fullname = Column(String)
    nickname = Column(String)

    def __repr__(self):
        return "<User(name='%s', fullname='%s', nickname='%s')>" % (self.name, self.fullname, self.nickname)


In [8]:
### Create schema ###

Base.metadata.drop_all(bind=engine)
print('before creating, tables:', engine.table_names())
Base.metadata.create_all(engine)
print('after creating, tables:', engine.table_names())

2019-05-12 07:42:14,621 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-05-12 07:42:14,623 INFO sqlalchemy.engine.base.Engine ()
2019-05-12 07:42:14,626 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-05-12 07:42:14,627 INFO sqlalchemy.engine.base.Engine ()
2019-05-12 07:42:14,630 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("users")
2019-05-12 07:42:14,631 INFO sqlalchemy.engine.base.Engine ()
2019-05-12 07:42:14,634 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2019-05-12 07:42:14,636 INFO sqlalchemy.engine.base.Engine ()
before creating, tables: []
2019-05-12 07:42:14,638 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("users")
2019-05-12 07:42:14,639 INFO sqlalchemy.engine.base.Engine ()
2019-05-12 07:42:14,641 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE users (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	fullname

In [9]:
### Insert ##

ed_user = User(name='ed', fullname='Ed Jones', nickname='edsnickname')
session.add(ed_user)
session.commit()
our_user = session.query(User).filter_by(name='ed').all()
print(our_user)

2019-05-12 07:42:28,463 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-05-12 07:42:28,466 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2019-05-12 07:42:28,467 INFO sqlalchemy.engine.base.Engine ('ed', 'Ed Jones', 'edsnickname')
2019-05-12 07:42:28,469 INFO sqlalchemy.engine.base.Engine COMMIT
2019-05-12 07:42:28,471 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-05-12 07:42:28,473 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name = ?
2019-05-12 07:42:28,474 INFO sqlalchemy.engine.base.Engine ('ed',)
[<User(name='ed', fullname='Ed Jones', nickname='edsnickname')>]


In [10]:
### Rollback ##

ed_user = User(name='ed', fullname='Ed Mickey', nickname='edsnickname')
session.add(ed_user)
session.rollback()
users = session.query(User).filter_by(name='ed').all()
print(users)

2019-05-12 07:42:33,760 INFO sqlalchemy.engine.base.Engine ROLLBACK
2019-05-12 07:42:33,764 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-05-12 07:42:33,766 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name = ?
2019-05-12 07:42:33,768 INFO sqlalchemy.engine.base.Engine ('ed',)
[<User(name='ed', fullname='Ed Jones', nickname='edsnickname')>]


In [11]:
### Building a Relationship ##

class Address(Base):
    __tablename__ = 'addresses'

    id = Column(Integer, primary_key=True)
    email_address = Column(String, nullable=False)
    user_id = Column(Integer, ForeignKey('users.id'))
    user = relationship("User", back_populates="addresses")
    
    def __repr__(self):
        return "<Address(email_address='%s')>" % self.email_address
    
User.addresses = relationship("Address", order_by=Address.id, back_populates="user")
    
Base.metadata.create_all(engine)
print('tables:', engine.table_names())

2019-05-12 07:42:50,229 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("users")
2019-05-12 07:42:50,231 INFO sqlalchemy.engine.base.Engine ()
2019-05-12 07:42:50,232 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("addresses")
2019-05-12 07:42:50,233 INFO sqlalchemy.engine.base.Engine ()
2019-05-12 07:42:50,235 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE addresses (
	id INTEGER NOT NULL, 
	email_address VARCHAR NOT NULL, 
	user_id INTEGER, 
	PRIMARY KEY (id), 
	FOREIGN KEY(user_id) REFERENCES users (id)
)


2019-05-12 07:42:50,235 INFO sqlalchemy.engine.base.Engine ()
2019-05-12 07:42:50,237 INFO sqlalchemy.engine.base.Engine COMMIT
2019-05-12 07:42:50,238 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2019-05-12 07:42:50,239 INFO sqlalchemy.engine.base.Engine ()
tables: ['addresses', 'users']


In [12]:
### Join ###

jack = User(name='jack', fullname='Jack Bean', nickname='gjffdd')
print('jack addresses:', jack.addresses)

jack.addresses = [Address(email_address='jack@google.com'),
                  Address(email_address='j25@yahoo.com')]
session.add(jack)
session.commit()
user = session.query(User).filter_by(name='jack').one()
print(user.addresses)

jack addresses: []
2019-05-12 07:42:56,249 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2019-05-12 07:42:56,251 INFO sqlalchemy.engine.base.Engine ('jack', 'Jack Bean', 'gjffdd')
2019-05-12 07:42:56,254 INFO sqlalchemy.engine.base.Engine INSERT INTO addresses (email_address, user_id) VALUES (?, ?)
2019-05-12 07:42:56,255 INFO sqlalchemy.engine.base.Engine ('jack@google.com', 2)
2019-05-12 07:42:56,256 INFO sqlalchemy.engine.base.Engine INSERT INTO addresses (email_address, user_id) VALUES (?, ?)
2019-05-12 07:42:56,257 INFO sqlalchemy.engine.base.Engine ('j25@yahoo.com', 2)
2019-05-12 07:42:56,258 INFO sqlalchemy.engine.base.Engine COMMIT
2019-05-12 07:42:56,260 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-05-12 07:42:56,261 INFO sqlalchemy.engine.base.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name = ?
2019-05-12